In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv',index_col='Id')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv',index_col='Id')

train.shape, test.shape

In [ ]:
train.head()

In [ ]:
missing = train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [ ]:
import scipy.stats as st
y = train['SalePrice']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)

In [ ]:
quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
quantitative.remove('SalePrice')

qualitative = [f for f in train.columns if train.dtypes[f] == 'object']

f = pd.melt(train, value_vars=quantitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

In [ ]:
for c in qualitative:
    train[c] = train[c].astype('category')
    if train[c].isnull().any():
        train[c] = train[c].cat.add_categories(['MISSING'])
        train[c] = train[c].fillna('MISSING')

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
f = pd.melt(train, id_vars=['SalePrice'], value_vars=qualitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "SalePrice")

In [ ]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
    
qual_encoded = []
for q in qualitative:  
    encode(train, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)

def pairplot(x, y, **kwargs):
    ax = plt.gca()
    ts = pd.DataFrame({'time': x, 'val': y})
    ts = ts.groupby('time').mean()
    ts.plot(ax=ax)
    plt.xticks(rotation=90)
    
f = pd.melt(train, id_vars=['SalePrice'], value_vars=quantitative+qual_encoded)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(pairplot, "value", "SalePrice")

There are lots of nonlinearities this may be the cause why some variables wont be selected by Lasso/Lars. Some factors like YearBuilt, 1stFlrSF, 2ndFlrSF, Neighborhood_E look like they would benefit from adding quadratic term to regression. But on the other hand this will most probably provoke overfit.

# Price Segments
It is possible that correlations shift with change of SalePrice.

In [ ]:
features = quantitative

standard = train[train['SalePrice'] < 200000]
pricey = train[train['SalePrice'] >= 200000]

diff = pd.DataFrame()
diff['feature'] = features
diff['difference'] = [(pricey[f].fillna(0.).mean() - standard[f].fillna(0.).mean())/(standard[f].fillna(0.).mean())
                      for f in features]

sns.barplot(data=diff, x='feature', y='difference')
x=plt.xticks(rotation=90)

# Clustering

In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
features = quantitative + qual_encoded
model = TSNE(n_components=2, random_state=0, perplexity=50)
X = train[features].fillna(0.).values
tsne = model.fit_transform(X)

std = StandardScaler()
s = std.fit_transform(X)
pca = PCA(n_components=30)
pca.fit(s)
pc = pca.transform(s)
kmeans = KMeans(n_clusters=5)
kmeans.fit(pc)

fr = pd.DataFrame({'tsne1': tsne[:,0], 'tsne2': tsne[:, 1], 'cluster': kmeans.labels_})
sns.lmplot(data=fr, x='tsne1', y='tsne2', hue='cluster', fit_reg=False)
print(np.sum(pca.explained_variance_ratio_))

In [ ]:
# Concat the train and test
df = pd.concat((train,test))
df.shape

# Exploratory Data Analysis (EDA)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.select_dtypes(include=['int64','float64']).columns

In [ ]:
df.select_dtypes(include=['object']).columns

In [ ]:
#Show the null values using heatmap
plt.figure(figsize=(16,9))
sns.heatmap(df.isnull())

These white cells are showing the null values

In [ ]:
# Get the percentages of null value
null_percent = df.isnull().sum()/df.shape[0]*100
null_percent

In [ ]:
col_for_drop = null_percent[null_percent > 20].keys() # if the null value % 20 or > 20 so need to drop it
# drop columns
df = df.drop(col_for_drop, "columns")
df.shape

In [ ]:
# find the unique value count
for i in df.columns:
    print(i + "\t" + str(len(df[i].unique())))

In [ ]:
# Plot the distplot of target
plt.figure(figsize=(10,8))
bar = sns.distplot(train["SalePrice"])
bar.legend(["Skewness: {:.2f}".format(train['SalePrice'].skew())])

In [ ]:
# correlation heatmap
plt.figure(figsize=(25,25))
ax = sns.heatmap(train.corr(), cmap = "coolwarm", annot=True, linewidth=2)

# to fix the bug "first and last row cut in half of heatmap plot"
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
# correlation heatmap of higly correlated features with SalePrice
hig_corr = train.corr()
hig_corr_features = hig_corr.index[abs(hig_corr["SalePrice"]) >= 0.3]
print(hig_corr_features)

plt.figure(figsize=(10,8))
ax = sns.heatmap(train[hig_corr_features].corr(), cmap = "coolwarm", annot=True, linewidth=3)
# to fix the bug "first and last row cut in half of heatmap plot"
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
# Plot regplot to get the nature of highly correlated data
plt.figure(figsize=(16,9))
for i in range(len(hig_corr_features)):
    if i <= 9:
        plt.subplot(3,4,i+1)
        plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
        sns.regplot(data=train, x = hig_corr_features[i], y = 'SalePrice')

# Handling Missing Value

Handling missing value of Bsmt feature

In [ ]:
df.isnull().sum()

In [ ]:
df['LotFrontage'].fillna(np.mean(df['LotFrontage']), inplace=True)

In [ ]:
def fill_null(values):
    type = values[0]
    area = values[1]
    if pd.isnull(type):
        return "None",0
    else:
        return values

df[['MasVnrType','MasVnrArea']] = df[['MasVnrType','MasVnrArea']].apply(fill_null, axis=1)

In [ ]:
df['BsmtQual'].fillna(df['BsmtQual'].mode()[0], inplace=True)
df['BsmtCond'].fillna(df['BsmtCond'].mode()[0], inplace=True)
df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0], inplace=True)
df['BsmtFinType1'].fillna(df['BsmtFinType1'].mode()[0], inplace=True)
df['BsmtFinType2'].fillna(df['BsmtFinType2'].mode()[0], inplace=True)
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace=True)

In [ ]:
df['GarageType'].fillna('No',inplace=True) # fill no that doens't have any garage type
df['GarageYrBlt'].fillna(0,inplace=True) # fill 0 that doesn't have any garage yr build because they don't have any garage
df['GarageFinish'].fillna('No',inplace=True)
df['GarageQual'].fillna('No',inplace=True) 
# GarageCond is similar to GarageQual so no point in storing the same columns, we will drop it later.df
# Exterior2nd is similar to Exterior1st we will also drop it later

In [ ]:
df.drop('GarageCond', axis=1, inplace=True)
df.drop('Exterior2nd', axis=1, inplace=True)

Handeled the remaining null features

In [ ]:
df['MSZoning'].fillna(df['MSZoning'].mode()[0], inplace=True)
df['Utilities'].fillna(df['Utilities'].mode()[0], inplace=True)
df['Exterior1st'].fillna(df['Exterior1st'].mode()[0], inplace=True)
df['BsmtFinSF1'].fillna(df['BsmtFinSF1'].mean(), inplace=True)
df['BsmtFinSF2'].fillna(df['BsmtFinSF2'].mean(), inplace=True)
df['BsmtUnfSF'].fillna(df['BsmtUnfSF'].mean(), inplace=True)
df['TotalBsmtSF'].fillna(df['TotalBsmtSF'].mean(), inplace=True)
df['BsmtFullBath'].fillna(df['BsmtFullBath'].mode()[0], inplace=True)
df['BsmtHalfBath'].fillna(df['BsmtHalfBath'].mode()[0], inplace=True)
df['KitchenQual'].fillna(df['KitchenQual'].mode()[0], inplace=True)
df['Functional'].fillna(df['Functional'].mode()[0], inplace=True)
df['GarageCars'].fillna(df['GarageCars'].mode()[0], inplace=True)
df['GarageArea'].fillna(df['GarageArea'].mean(), inplace=True)
df['SaleType'].fillna(df['SaleType'].mode()[0], inplace=True)


In [ ]:
df.isnull().sum()

Now convert all object columns into numeric column

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns
object_columns

In [ ]:
for i in object_columns:
    df = pd.get_dummies(df, columns=[i])

In [ ]:
df.info()

# scaling dataset with robust scaler as we know that there might be some outliers within the dataset.

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(df)
df = scaler.transform(df)

# Machine Learning Model Building

In [ ]:
train_len = len(train)

In [ ]:
X_train = df[:train_len]
X_test = df[train_len:]
y_train = train.SalePrice

X_train.shape, X_test.shape, y_train.shape

# Cross Validation

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, r2_score

def test_model(model, X_train=X_train, y_train=y_train):
    cv = KFold(n_splits = 3, shuffle=True, random_state = 45) # Split your data to k Folds
    r2 = make_scorer(r2_score) # r2_score is the R^2 (coefficient of determination) which used to evaluate the performance of linear Regression Model
    # formula is R^2 = 1-SS(res)/SS(tot)
    r2_val_score = cross_val_score(model, X_train, y_train, cv=cv, scoring = r2) # Evaluates the data and returns the score
    score = [r2_val_score.mean()]
    return score

# Linear Regression

In [ ]:
import sklearn.linear_model as linear_model
LR = linear_model.LinearRegression()
test_model(LR)

# L2, Ridge regression

In [ ]:
rdg = linear_model.Ridge()
test_model(rdg)

# L1, Lasso Regression

In [ ]:
lasso = linear_model.Lasso(alpha=1e-4)
test_model(lasso)

# Support Vector Machine

In [ ]:
from sklearn.svm import SVR
svr_reg = SVR(kernel= 'rbf')
test_model(svr_reg)

We got so low accuracy let's hyperparameter tuned this model. 

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV 
# params = {'kernel': ['linear', 'rbf', 'sigmoid'], 
#           'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#           'C': [0.1, 1, 10, 100, 1000], 
#           'epsilon': [1, 0.2, 0.1, 0.01, 0.001, 0.0001]}

# rand_search = RandomizedSearchCV(svr_reg, param_distributions=params, n_jobs=-1, cv=11) 
# rand_search.fit(X_train, y_train) 
# rand_search.best_params_

Note: It took so much time, that's why I comment it. These are the parameters I got from this hyperparameter tuning

In [ ]:
svr_reg= SVR(kernel='rbf', C=1000, epsilon=0.1, gamma=0.01)
test_model(svr_reg)

Still we didn't get much better accuracy

# Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(random_state=21)
test_model(dt_reg)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators = 1000, random_state=51)
test_model(rf_reg)

# Bagging and Boosting

In [ ]:
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
br_reg = BaggingRegressor(n_estimators=1000, random_state=51)
gbr_reg = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, loss='ls', random_state=51)

In [ ]:
test_model(br_reg)

In [ ]:
test_model(gbr_reg)

# XG Boost

In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
test_model(xgb_reg)

We got better accuracy with gradient boosting regression till now, let's submit this prediction with this as of now. 

# Submitting Prediction

In [ ]:
gbr_reg.fit(X_train, y_train)
y_pred = gbr_reg.predict(X_test).round(2)
y_pred

In [ ]:
ID = pd.Series(test.index)
ID

In [ ]:
submit_result = pd.concat([ID, pd.DataFrame(y_pred)], axis=1)
submit_result.columns=['Id', 'SalePrice']
submit_result

In [ ]:
submit_result.to_csv('submission.csv', index=False)
submit_result